In [27]:
import pandas as pd 
import json 
import os

Fichier "summarizedActivities.json" : 
Ce fichier contient des objets json qui représente à chaque fois une activité 

In [28]:
#Cette fonction permet de récupérer les features choisis dans les données des activités
#Elle prend en entrée le fichier json des activités d'un utilisateur et un nom pour le fichier csv de sortie afin d'assurer la tracabilité.
def Summarized(fichier, nom : str):
    with open(fichier) as summarized:
        data =json.load(summarized)
    data_list = []

    for entry in data:
        for i in range(len(entry["summarizedActivitiesExport"])):
            #activity_type
            activity_type = entry["summarizedActivitiesExport"][i]['activityType']
            #begintimestamp
            if "beginTimestamp" in entry["summarizedActivitiesExport"][i]:
                beginTimestamp = entry["summarizedActivitiesExport"][i]['beginTimestamp']
            else:
                beginTimestamp = None
            #duration
            duration = entry["summarizedActivitiesExport"][i]['duration']
            #startLatitude
            if "startLatitude" in entry["summarizedActivitiesExport"][i]:
                startLatitude = entry["summarizedActivitiesExport"][i]["startLatitude"]
            else:
                startLatitude = None
            #startLongitude
            if "startLongitude" in entry["summarizedActivitiesExport"][i]:
                startLongitude = entry["summarizedActivitiesExport"][i]["startLongitude"]
            else:
                startLongitude = None




            data_list.append({'activity_type': activity_type, 'begintimestamp': beginTimestamp, 'duration': duration,  
                            'startLatitude': startLatitude, 'startLongitude': startLongitude

                        
                        })
        
    df_activity = pd.DataFrame(data_list)

    df_activity['begintimestamp'] = pd.to_datetime(df_activity['begintimestamp'], unit='ms') #UTC si jaja
    df_activity['duration'] = pd.to_timedelta(df_activity['duration'], unit='ms')
    df_activity['endtimestamp'] = df_activity['begintimestamp'] + pd.to_timedelta(df_activity['duration'])
    df_activity['endtimestamp'] = df_activity['endtimestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_activity['Date'] = df_activity['begintimestamp'].dt.strftime('%Y-%m-%d')
    df_activity['duration'] = df_activity['duration'].dt.total_seconds().apply(lambda x: pd.to_datetime(x, unit='s').strftime('%H:%M:%S'))
    
    file_path = '/Users/thibaut/Desktop/Garmin_Project_ML/Data/Activities/'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    df_activity.to_csv(f"{file_path}ACTIVITY_data_{nom}.csv", index=False)
    


In [29]:
# #Cette fonction permet de récupérer les features choisis dans les données des activités
# #Elle prend en entrée le fichier json des activités d'un utilisateur et un nom pour le fichier csv de sortie afin d'assurer la tracabilité.
# def Summarized(fichier, nom : str):
#     with open(fichier) as summarized:
#         data =json.load(summarized)
#     data_list = []

#     for entry in data:
#         for activity in entry["summarizedActivitiesExport"]:
#             #activityType
#             activity_type = activity.get['activityType', None]
#             #beginTimestamp
#             beginTimestamp = activity.get('beginTimestamp', None)
#             #duration
#             duration = activity.get['duration', None]
#             #startLatitude
#             startLatitude = activity.get('startLatitude', None)
#             #startLongitude
#             startLongitude = activity.get('startLongitude', None)
            
#             #Ajoute les données dans la liste
#             data_list.append({'activity_type': activity_type, 'begintimestamp': beginTimestamp, 'duration': duration,  
#                             'startLatitude': startLatitude, 'startLongitude': startLongitude

                        
#                         })
#     #Transforme la liste en dataframe
#     df_activity = pd.DataFrame(data_list)
#     #Quelques pré-traitements ici qui permettent d'avoir les données au même format que dans les autres fichiers de données (Sleep/UDS).
#     #Notamment au niveau de la date car c'est sur ce feature que les dataframes seront merged.
#     df_activity['begintimestamp'] = pd.to_datetime(df_activity['begintimestamp'], unit='ms') #UTC si jaja
#     df_activity['duration'] = pd.to_timedelta(df_activity['duration'], unit='ms')
#     df_activity['endtimestamp'] = df_activity['begintimestamp'] + pd.to_timedelta(df_activity['duration'])
#     df_activity['endtimestamp'] = df_activity['endtimestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
#     df_activity['Date'] = df_activity['begintimestamp'].dt.strftime('%Y-%m-%d')
#     df_activity['duration'] = df_activity['duration'].dt.total_seconds().apply(lambda x: pd.to_datetime(x, unit='s').strftime('%H:%M:%S'))
#     #Chemin du dossier pour stocker les fichiers exportés 
#     file_path = '/Users/thibaut/Desktop/Garmin_Project_ML/Data/Activities/'
#     if not os.path.exists(file_path):
#         os.makedirs(file_path)
#     df_activity.to_csv(f"{file_path}ACTIVITY_data_{nom}.csv", index=False)

Il n'y a que 1 fichier d'activités par utilisateur donc un appel à la fonction par utilisateur suffit

In [30]:
fichier_T = "/Users/thibaut/Desktop/Garmin_Project_ML/Data/Garmin_T/DI_CONNECT/DI-Connect-Fitness/thibaut.frabb@gmail.com_0_summarizedActivities.json"
Summarized(fichier_T, "T")


In [31]:
fichier_M = "/Users/thibaut/Desktop/Garmin_Project_ML/Data/Garmin_M/DI_CONNECT/DI-Connect-Fitness/maxime.frab@gmail.com_0_summarizedActivities.json"
Summarized(fichier_M, "M")

In [32]:
fichier_G = "/Users/thibaut/Desktop/Garmin_Project_ML/Data/Garmin_G/DI_CONNECT/DI-Connect-Fitness/bag.sava11@gmail.com_0_summarizedActivities.json"
Summarized(fichier_G, "G")

In [33]:
fichier_D = "/Users/thibaut/Desktop/Garmin_Project_ML/Data/Garmin_D/DI_CONNECT/DI-Connect-Fitness/challandesdounia@gmail.com_0_summarizedActivities.json"
Summarized(fichier_D, "D")

Module qui permettait de transformer les coordonnées GPS en une adresse mais le feature des coordonnées sera utilisé finalement

In [34]:
# from geopy.geocoders import Nominatim
# from geopy.point import Point

# df_activity['Point'] = df_activity['GPS'].apply(lambda x: Point(x) if x is not None else None)
# geolocator = Nominatim(user_agent="my_geocoder")
# df_activity['address'] = df_activity['Point'].apply(lambda x: geolocator.reverse(x).address if x is not None else None)

